# Accessing ACLED Data Programmatically by Date_Range and/or Region

ACLED API documentation: https://acleddata.com/acleddatanew/wp-content/uploads/dlm_uploads/2019/01/API-User-Guide2020.pdf


#### The notebook is designed to programmatically pull ACLED data without concerns of data caps or taxing the API.

#### The API has a standard 500 data point limit but can be expanded to encompass all the data.  This notebook does this by identifying the total data points in ACLED before any queries are done using the Regions portion of the API.  If the notebook user is interested in ACLED globally the total data point count will be used as the limit parameter. This allows notebook users to only perform one pull.

#### If the notebook user is interested in a specific region it also uses the region portion of the API to pull the total data point counts for that region.  The limit parameter then becomes the total count for the specified region.  

#### The notebook then pushes it into a pandas dataframe and provides the user an option of they want to save it off as a csv.


# Info for Variables
- start_date and end_date
    - The time range you want to search on.  Formatting is YYYY-MM-DD 
- specific_reg (case sensitive)
    - Yes - if you are looking at specific region
    - No - if you aren't
- reg
    - If you selected Yes for spec_reg, this is the specific region you want to pull for. In reg variable you need to input the numeric code associated with the region below. 
        - 1 - Western Africa
        - 2 - Middle Africa
        - 3 - Eastern Africa
        - 4 - Southern Africa
        - 5 - Northern Africa
        - 7 - Southern Asia
        - 9 - South-Eastern Asia
        - 11 - Middle East
        - 12 - Europe
        - 13 - Caucasus and Central Asia
        - 14 - Central America
        - 15 - South America
        - 16 - Caribbean
- csv_create
    - Yes - if you want to save off the data
    - No - if you don't want to save the data but keeping working
- csv_file
    - entire file name making sure it ends with a .csv (ex. Tiffany_test.csv)
    
* Note - Make sure that all variables have quotes around them (example: '12')

# Populate Variables

In [10]:
##Variables
start_date = '2019-03-01'
end_date = '2020-03-14'
specific_reg = 'Yes'
reg = '16'
csv_create = 'No'
csv_file = 'Caribbean.csv'

### Shouldn't have to change anything in the cells below

In [11]:
##import libraries and packages
import requests
import pandas as pd
from pandas.io.json import json_normalize 

In [12]:
##access regional counts
URL = "https://api.acleddata.com/region/read?terms=accept"
r = requests.get(url = URL)
data = r.json()
df_reg = json_normalize(data, 'data')

In [13]:
##get total count of events worldwide 
df_reg['count'] = df_reg['event_count'].astype(int)
total = df_reg['count'].sum(axis=0)
##create dictionary for region numeric code and counts of events in each region
dictionary_lim = pd.Series(df_reg.event_count.values, index = df_reg.region).to_dict()


In [14]:
##If doing a global query will limit to total global count
##If doing a regional query will limit to total regional count
if specific_reg == 'No':
    lim = str(total)
    print(lim)
if specific_reg == 'Yes':
    lim = dictionary_lim[reg] 
    print(lim)

1479


In [15]:
##Building of Parameters  - Should not need to change
URL = 'https://api.acleddata.com/acled/read?terms=accept'
if specific_reg == 'No':
    PARAMS = "&limit="+lim+"&event_date="+start_date+"|"+end_date+"&event_date_where=BETWEEN"
elif specific_reg == 'Yes':
    PARAMS = "&limit="+lim+"&region="+reg+"&event_date="+start_date+"|"+end_date+"&event_date_where=BETWEEN"

In [16]:
##Request event data
r = requests.get(url = URL, params = PARAMS)
data_event = r.json()
df_event = json_normalize(data_event, 'data')

In [17]:
df_event.count()

##option to save data off as a csv or keep working
if csv_create == 'Yes':
    df_event.to_csv(csv_file)
elif csv_create == 'No':
    print('Keep on analyzing your pandas dataframe you crazy data person!')

Keep on analyzing your pandas dataframe you crazy data person!


In [18]:
df_event

,actor1,actor2,admin1,admin2,admin3,assoc_actor_1,assoc_actor_2,country,data_id,event_date,...,location,longitude,notes,region,source,source_scale,sub_event_type,time_precision,timestamp,year
0,Protesters (Dominican Republic),,Santo Domingo,Santo Domingo Este,,MINPC: Nigua Independent Movement for Change; ...,,Dominican Republic,6992669,2020-03-13,...,Santo Domingo,-69.8923,"On 13 March 2020, in Santo Domingo representat...",Caribbean,El Nacional (Dominican Republic),National,Peaceful protest,1,1584380267,2020
1,Protesters (Barbados),,Saint Michael,,,,,Barbados,6992752,2020-03-13,...,Bridgetown,-59.6202,"On 13 March in Bridgetown, Saint Michael, a sm...",Caribbean,Kaieteur News,International,Peaceful protest,1,1584380267,2020
2,Rioters (Barbados),Civilians (Barbados),Saint Michael,,,,,Barbados,6992837,2020-03-13,...,Bridgetown,-59.6202,"On 13 March 2020, in Bridgetown, Sain Michael,...",Caribbean,Loop News Barbados,National,Mob violence,1,1584380267,2020
3,Police Forces of Cuba (1959-),Civilians (Cuba),Santiago de Cuba,Santiago de Cuba,,,UNPACU: Patriotic Union of Cuba,Cuba,6992641,2020-03-12,...,Santiago de Cuba,-75.8267,"On 12 March 2020, in Santiago de Cuba, police ...",Caribbean,CubaNet,National,Attack,1,1584380267,2020
4,Protesters (Cuba),,Villa Clara,Santa Clara,,,,Cuba,6992643,2020-03-12,...,Santa Clara,-79.9647,"On 12 March 2020, in Santa Clara, Villa Clara,...",Caribbean,CubaNet,National,Peaceful protest,1,1584380267,2020
5,Unidentified Armed Group (Puerto Rico),Civilians (Puerto Rico),San Juan,,,,,Puerto Rico,6992718,2020-03-12,...,San Juan,-66.1057,"On 12 March 2020, in San Juan, Puerto Rico, tw...",Caribbean,El Vocero de Puerto Rico,National,Attack,1,1584380267,2020
6,Police Forces of Haiti (2011-),Civilians (Haiti),Ouest,Port-au-Prince,Delmas,,Government of Haiti (2011-),Haiti,6992836,2020-03-12,...,Delmas,-72.3028,"On March 12, 2020, in Delmas, Port-au-Prince, ...",Caribbean,Le Nouvelliste,National,Attack,1,1584380267,2020
7,Protesters (Dominican Republic),Police Forces of Dominican Republic (2004-),Santo Domingo,Santo Domingo Este,,Health Workers (Dominican Republic),,Dominican Republic,6992668,2020-03-11,...,Santo Domingo,-69.8923,"On 11 March 2020, in Santo Domingo medical sta...",Caribbean,El Nacional (Dominican Republic),National,Protest with intervention,1,1584380267,2020
8,Unidentified Armed Group (Trinidad and Tobago),Civilians (Trinidad and Tobago),Tunapuna/Piarco,,,,,Trinidad and Tobago,6992937,2020-03-11,...,Maracas,-61.4197,"On 11 March 2020, in Maracas, Tunapuna/Piarco,...",Caribbean,Trinidad and Tobago Guardian,National,Attack,1,1584380267,2020
9,Rioters (Haiti),,Ouest,Port-au-Prince,Port-au-Prince,,,Haiti,6992626,2020-03-11,...,Port-au-Prince,-72.3388,"On 11 March 2020, south of Port-au-Prince, dem...",Caribbean,Alterpresse,National,Violent demonstration,1,1584380267,2020
